In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier

# warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# get dataset

# train data
dtr = pd.read_csv('~/Documents/ML-USU-SP21-main/dataset/dsn-prebootcamp-2021/train.csv')
X_tr = dtr.drop(['Customer_ID','Certification','Effective To Date','Active_Policy','Sales Channel','Best_Offer'],axis = 1)
y_tr = pd.DataFrame(dtr['Best_Offer'])

# test data
dte = pd.read_csv('~/Documents/ML-USU-SP21-main/dataset/dsn-prebootcamp-2021/test.csv')
X_te = dte.drop(['Customer_ID','Certification','Effective To Date','Active_Policy','Sales Channel'],axis = 1)
# sum(dtr.duplicated())
# sum(dte.duplicated())

In [3]:
# dtr['Effective To Date'] = pd.to_datetime(dtr['Effective To Date'])
# dte['Effective To Date'] = pd.to_datetime(dte['Effective To Date'])
# dtr['Effective To Date'].now()

In [4]:
Le = LabelEncoder()

# train dataset

X_tr['Sex'] = Le.fit_transform(X_tr['Sex'])
X_tr['Region_Code'] = Le.fit_transform(X_tr['Region_Code'])
X_tr['Marital Status'] = Le.fit_transform(X_tr['Marital Status'])
X_tr['State_of_Origin'] = Le.fit_transform(X_tr['State_of_Origin'])
X_tr['Customer_Reply'] = Le.fit_transform(X_tr['Customer_Reply'])
X_tr['Plan'] = Le.fit_transform(X_tr['Plan'])
X_tr['Status_of_Employment'] = Le.fit_transform(X_tr['Status_of_Employment'])
X_tr['Vehicle Class'] = Le.fit_transform(X_tr['Vehicle Class'])
X_tr['Vehicle Size'] = Le.fit_transform(X_tr['Vehicle Size'])
X_tr['kind_of_policy'] = Le.fit_transform(X_tr['kind_of_policy'])

# test dataset

X_te['Sex'] = Le.fit_transform(X_te['Sex'])
X_te['Region_Code'] = Le.fit_transform(X_te['Region_Code'])
X_te['Marital Status'] = Le.fit_transform(X_te['Marital Status'])
X_te['State_of_Origin'] = Le.fit_transform(X_te['State_of_Origin'])
X_te['Customer_Reply'] = Le.fit_transform(X_te['Customer_Reply'])
X_te['Plan'] = Le.fit_transform(X_te['Plan'])
X_te['Status_of_Employment'] = Le.fit_transform(X_te['Status_of_Employment'])
X_te['Vehicle Class'] = Le.fit_transform(X_te['Vehicle Class'])
X_te['Vehicle Size'] = Le.fit_transform(X_te['Vehicle Size'])
X_te['kind_of_policy'] = Le.fit_transform(X_te['kind_of_policy'])

In [10]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)

In [11]:
pd.crosstab(y_tr['Best_Offer'], y_tr['Best_Offer'], normalize='all') * 100

Best_Offer,0,1,2,3
Best_Offer,,,,
0,25.0,0.0,0.0,0.0
1,0.0,25.0,0.0,0.0
2,0.0,0.0,25.0,0.0
3,0.0,0.0,0.0,25.0


In [4]:
cat = CatBoostClassifier()
pipe = Pipeline([
    ('scale' , StandardScaler()),
    ('model' , cat)
])
pipe.get_params()
mod = pipe.fit(X_tr, y_tr)
y = pd.DataFrame(mod.predict(X_te))
yy = pd.DataFrame(mod.predict_proba(X_te))
yy

Learning rate set to 0.086979
0:	learn: 1.3589955	total: 296ms	remaining: 4m 55s
1:	learn: 1.3346658	total: 308ms	remaining: 2m 33s
2:	learn: 1.3112043	total: 319ms	remaining: 1m 46s
3:	learn: 1.2902819	total: 330ms	remaining: 1m 22s
4:	learn: 1.2728727	total: 341ms	remaining: 1m 7s
5:	learn: 1.2576914	total: 353ms	remaining: 58.5s
6:	learn: 1.2451036	total: 365ms	remaining: 51.8s
7:	learn: 1.2351480	total: 379ms	remaining: 47s
8:	learn: 1.2242338	total: 391ms	remaining: 43.1s
9:	learn: 1.2150353	total: 403ms	remaining: 39.9s
10:	learn: 1.2059927	total: 414ms	remaining: 37.2s
11:	learn: 1.1984667	total: 426ms	remaining: 35.1s
12:	learn: 1.1919692	total: 437ms	remaining: 33.2s
13:	learn: 1.1856019	total: 449ms	remaining: 31.6s
14:	learn: 1.1805948	total: 460ms	remaining: 30.2s
15:	learn: 1.1749930	total: 472ms	remaining: 29s
16:	learn: 1.1699488	total: 483ms	remaining: 27.9s
17:	learn: 1.1659902	total: 510ms	remaining: 27.8s
18:	learn: 1.1623822	total: 526ms	remaining: 27.1s
19:	learn: 

,0,1,2,3
0,0.127094,0.484437,0.186119,0.202350
1,0.460746,0.359569,0.106640,0.073045
2,0.020849,0.930069,0.031947,0.017135
3,0.610397,0.140581,0.188895,0.060127
4,0.447023,0.348145,0.111823,0.093009
...,...,...,...,...
2736,0.106682,0.338784,0.060483,0.494051
2737,0.923502,0.063668,0.008703,0.004126
2738,0.247500,0.416790,0.030994,0.304717
2739,0.205994,0.405618,0.311638,0.076751


In [41]:
xgb = XGBClassifier(subsample=1,max_depth=3,eta=1)
pipe = Pipeline([
    ('scale' , StandardScaler()),
    ('model' , xgb)
])
pipe.get_params()
mod = pipe.fit(X_tr, y_tr)
y_te = pd.DataFrame(mod.predict(X_te))
y_te

[17:29:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,0
0,2
1,1
2,1
3,0
4,2
...,...
2736,1
2737,0
2738,3
2739,1


In [42]:
grid = GridSearchCV(
    estimator = pipe,
    param_grid = {
        'model__max_depth' : [0,1,2,3],
        'model__max_depth_step' : [0,1,2,3],
    },
    n_jobs = -1,
    verbose = 2,
    cv = 5
)
y1 = pd.DataFrame(grid.fit(X_tr, y_tr).predict(X_te))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[17:30:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "max_depth_step" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:30:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [43]:
grid.best_params_

{'model__max_depth': 3, 'model__max_depth_step': 0}

In [44]:
grid.best_estimator_

Pipeline(steps=[('scale', StandardScaler()),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eta=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='', learning_rate=1,
                               max_delta_step=0, max_depth=3, max_depth_step=0,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=4, num_parallel_tree=1,
                               objective='multi:softprob', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [45]:
df1 = pd.concat([dte['Customer_ID']], axis = 1)
df1['best_offer'] = y_te
df1.to_csv('submit3.csv',index=False)